In [348]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [ ]:
#The Wikipedia page was changed on November, 2019
#We will use the older version
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canad%D7%A9a:_M&oldid=926306543"
res = requests.get(url)

In [306]:
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))
df = data[data['Borough'] != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [314]:
#Grouping by Borough and Postcode
df = df.groupby(['Postcode','Borough'], as_index=False).agg(','.join)
df = df.sample(frac=1).reset_index(drop=True)
df

,Postcode,Borough,Neighbourhood
0,M6E,York,Caledonia-Fairbanks
1,M1E,Scarborough,"Guildwood,Morningside,West Hill"
2,M3J,North York,"Northwood Park,York University"
3,M4W,Downtown Toronto,Rosedale
4,M4M,East Toronto,Studio District
5,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
6,M3N,North York,Downsview Northwest
7,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
8,M5H,Downtown Toronto,"Adelaide,King,Richmond"
9,M2J,North York,"Fairview,Henry Farm,Oriole"


In [309]:
df.shape

(103, 3)

In [332]:
geospatial_url = "https://cocl.us/Geospatial_data"
latlondata = pd.read_csv(geospatial_url)

In [338]:
latlondata.columns=['Postcode','Latitude','Longitude']
latlondata.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [339]:

#Merging of the neighbourhood database and the geospatial data
data = pd.merge(df, latlondata, on='Postcode')

In [340]:
data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512
1,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
2,M3J,North York,"Northwood Park,York University",43.767980,-79.487262
3,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
4,M4M,East Toronto,Studio District,43.659526,-79.340923
